<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 4

###  Big Query, SQL, Classification

---

### The Data

There are 5 individual tables that have the information, contained in a Google BigQuery database.  The setup info for BigQuery is located on our DSI wiki.  You will have to query with SQL, the dataset in order to complete this project.

The tables, with cooresonding attributes that exist are:

### businesses
- business_id: unique business identifier
- name: name of the business
- review_count: number of reviews per business
- city: city business resides in
- stars: average rating
- categories: categories the business falls into (can be one or multiple)
- latitude
- longitude
- neighborhoods: neighborhoods business belongs to
- variable: "property" of the business (a tag)
- value: True/False for the property

### reviews
- user_id: unique user identifier
- review_id: unique review identifier
- votes.cool: how many thought the review was "cool"
- business_id: unique business id the review is for
- votes.funny: how many thought the review was funny
- stars: rating given
- date: date of review
- votes.useful: how many thought the review was useful
- ... 100 columns of counts of most common 2 word phrases that appear in reviews in this review

### users
- yelping_since: signup date
- compliments.plain: # of compliments "plain"
- review_count: # of reviews:
- compliments.cute: total # of compliments "cute"
- compliments.writer: # of compliments "writer"
- compliments.note: # of compliments "note" (not sure what this is)
- compliments.hot: # of compliments "hot" (?)
- compliments.cool: # of compliments "cool"
- compliments.profile: # of compliments "profile"
- average_stars: average rating
- compliments.more: # of compliments "more"
- elite: years considered "elite"
- name: user's name
- user_id: unique user id
- votes.cool: # of votes "cool"
- compliments.list: # of compliments "list"
- votes.funny: # of compliments "funny"
- compliments.photos: # of compliments "photos"
- compliments.funny: # of compliments "funny"
- votes.useful: # of votes "useful"

### checkins
- business_id: unique business identifier
- variable: day-time identifier of checkins (0-0 is Sunday 0:00 - 1:00am,  for example)
- value: # of checkins at that time

### tips
- user_id: unique user identifier
- business_id: unique business identifier
- likes: likes that the tip has
- date: date of tip
- ... 100 columns of counts of most common 2 word phrases that appear in tips in this tip


The reviews and tips datasets in particular have parsed "NLP" columns with counts of 2-word phrases in that review or tip (a "tip", it seems, is some kind of smaller review).

The user dataset has a lot of columns of counts of different compliments and votes. We're not sure whether the compliments or votes are by the user or for the user.

Full details about this dataset area located here:
https://bigquery.cloud.google.com/dataset/bigquery-dsi-dave:yelp_arizona

---


If you look at the website, or the full data, you'll see I have removed pieces of the data and cut it down quite a bit. This is to simplify it for this project. Specifically, business are limited to be in these cities:

- Phoenix
- Surprise
- Las Vegas
- Waterloo

Apparently there is a city called "Surprise" in Arizona. 

Businesses are also restricted to at least be in one of the following categories, because we thought the mix of them was funny:

- Airports
- Breakfast & Brunch
- Bubble Tea
- Burgers
- Bars
- Bakeries
- Breweries
- Cafes
- Candy Stores
- Comedy Clubs
- Courthouses
- Dance Clubs
- Fast Food
- Museums
- Tattoo
- Vape Shops
- Yoga
    
---

### Project requirements

**You will be performing 4 different sections of analysis, like in the last project.**

Remember that classification targets are categorical and regression targets are continuous variables.

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Load your dataset(s) / setup / configure GBQ connection

---

Information about this dataset is located here:


**If you haven't done so, setup a project with the Google developer portal, following the directions here: [Getting Started with BigQuery](https://github.com/ga-students/DSI-SF-4/wiki/Getting-Started-with-BigQuery)**

In [105]:
import pandas as pd
import seaborn as sns
import numpy as np #we needed to add this
import patsy #patsy doesn't get a cool nickname
from sklearn.preprocessing import StandardScaler #added
from sklearn.neighbors import KNeighborsClassifier #K-neighbors classifier added
from sklearn.grid_search import GridSearchCV      #GridsearchCv added
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.metrics import roc_curve, auc


%matplotlib inline

project_id = "eternal-respect-151718"

sql = """
SELECT * FROM [bigquery-dsi-dave:yelp_arizona.reviews] 
LIMIT 2500
"""

df = pd.read_gbq(sql, project_id=project_id)

Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
Got 2500 rows.

Total time taken 5.27 s.
Finished at 2016-12-20 08:18:45.


In [3]:
df.columns

Index([u'user_id', u'review_id', u'votes_cool', u'business_id', u'votes_funny',
       u'stars', u'date', u'votes_useful', u'minutes_10', u'minutes_15',
       ...
       u'service_great', u'staff_friendly', u'super_friendly', u'sweet_potato',
       u'tasted_like', u'time_vegas', u'try_place', u've_seen', u've_tried',
       u'wait_staff'],
      dtype='object', length=108)

In [13]:
#Let's look at the data. So here we see a lot of superfluous information, but not enough relevant info. 
df.head(3)

,user_id,review_id,votes_cool,business_id,votes_funny,stars,date,votes_useful,minutes_10,minutes_15,...,service_great,staff_friendly,super_friendly,sweet_potato,tasted_like,time_vegas,try_place,ve_seen,ve_tried,wait_staff
0,-ZWUL1xfFP-WA9iS87S8hQ,UOQie6_QAJ8hjceRYk5BXg,0,P7wKlbSRsoRrfH3DFOY08A,0,1,2014-09-08,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,lqXbPv5vlD5fu0TU_71AGQ,pb0yWYEyXqe80w-rf69NjA,0,oCA2OZcd_Jo_ggVmUx3WVw,0,1,2014-11-28,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,vW8aeiByI-cAiamhenoSYA,RAgW45ub6iaFyZPCnI2mdw,0,hzUOqxpcnYYBgLlrgvjQYQ,0,1,2011-10-24,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [27]:
sql = """
SELECT * FROM [bigquery-dsi-dave:yelp_arizona.businesses] 

"""

biz_df = pd.read_gbq(sql, project_id=project_id)

Requesting query... ok.
Query running...
Query done.
Processed: 25.2 Mb

Retrieving results...
  Got page: 1; 21.0% done. Elapsed 9.12 s.
  Got page: 2; 42.0% done. Elapsed 15.71 s.
  Got page: 3; 62.0% done. Elapsed 20.46 s.
  Got page: 4; 83.0% done. Elapsed 25.87 s.
  Got page: 5; 100.0% done. Elapsed 30.55 s.
Got 152832 rows.

Total time taken 35.23 s.
Finished at 2016-12-20 07:30:31.


In [28]:
biz_df.head(1) #The business table has more relevant information

,business_id,name,review_count,city,stars,categories,latitude,longitude,neighborhoods,variable,value
0,XziiAJUc8cgj3McljtnYVw,Johnny Rockets,39,Las Vegas,3.0,"['Burgers', 'American (Traditional)', 'Sandwic...",36.114471,-115.169292,['The Strip'],attributes.Ambience.trendy,False


In [29]:
biz_df.isnull().sum() #Just checking to make sure there are no null entries

business_id      0
name             0
review_count     0
city             0
stars            0
categories       0
latitude         0
longitude        0
neighborhoods    0
variable         0
value            0
dtype: int64

In [30]:
biz_df.info() #checking what data types and counts we're dealing with

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152832 entries, 0 to 152831
Data columns (total 11 columns):
business_id      152832 non-null object
name             152832 non-null object
review_count     152832 non-null int64
city             152832 non-null object
stars            152832 non-null float64
categories       152832 non-null object
latitude         152832 non-null float64
longitude        152832 non-null float64
neighborhoods    152832 non-null object
variable         152832 non-null object
value            152832 non-null bool
dtypes: bool(1), float64(3), int64(1), object(6)
memory usage: 11.8+ MB


In [31]:
#Checking other tables
sql = """
SELECT * FROM [bigquery-dsi-dave:yelp_arizona.users] 

"""

users_df = pd.read_gbq(sql, project_id=project_id)

Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
  Got page: 2; 23.0% done. Elapsed 11.89 s.
  Got page: 3; 35.0% done. Elapsed 16.99 s.
  Got page: 4; 46.0% done. Elapsed 21.36 s.
  Got page: 5; 58.0% done. Elapsed 25.83 s.
  Got page: 6; 69.0% done. Elapsed 30.33 s.
  Got page: 7; 81.0% done. Elapsed 35.39 s.
  Got page: 8; 92.0% done. Elapsed 40.71 s.
  Got page: 9; 100.0% done. Elapsed 43.85 s.
Got 144206 rows.

Total time taken 50.55 s.
Finished at 2016-12-20 07:32:06.


In [32]:
sql = """
SELECT * FROM [bigquery-dsi-dave:yelp_arizona.tips] 
LIMIT 2500
"""

tips_df = pd.read_gbq(sql, project_id=project_id)

Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
Got 2500 rows.

Total time taken 6.58 s.
Finished at 2016-12-20 07:32:13.


In [56]:
#We changed the format of the business table to wide 
biz_wide = pd.pivot_table(biz_df, index=['business_id','city'], columns='variable', values='value').reset_index()
biz_wide.tail(2)

variable,business_id,city,attributes.Accepts Credit Cards,attributes.Accepts Insurance,attributes.Ages Allowed,attributes.Alcohol,attributes.Ambience.casual,attributes.Ambience.classy,attributes.Ambience.divey,attributes.Ambience.hipster,...,hours.Saturday.open,hours.Sunday.close,hours.Sunday.open,hours.Thursday.close,hours.Thursday.open,hours.Tuesday.close,hours.Tuesday.open,hours.Wednesday.close,hours.Wednesday.open,open
4130,zvvCrwrr4VGz-D9GtMhDGQ,Phoenix,True,None,None,False,False,False,False,False,...,None,None,None,None,None,None,None,None,None,True
4131,zwHM6hz8swKIG-4-BSKMPA,Phoenix,True,None,None,False,False,False,False,False,...,None,None,None,None,None,None,None,None,None,False


In [40]:
biz_wide_col = [col for col in biz_wide.columns if sum(biz_wide[col].isnull()) < 2000]

biz_wide = biz_wide[biz_wide_col]

#Ughh and now the cleaning begins...

In [42]:
biz_wide.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)
biz_wide.rename(columns=lambda x: x.replace('attributes', ''), inplace=True)
biz_wide.rename(columns=lambda x: x.replace('.', ''), inplace=True)
biz_wide.rename(columns=lambda x: x.replace('-', '_'), inplace=True)

In [43]:
biz_wide.info() #yikes, we're going to need to drop some values
                #I will resist dropping entire rows out of sheer laziness.
                #But I will drop useless variables

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4132 entries, 0 to 4131
Data columns (total 51 columns):
business_id             4132 non-null object
city                    4132 non-null object
Accepts_Credit_Cards    3896 non-null object
Alcohol                 3050 non-null object
Ambiencecasual          2873 non-null object
Ambienceclassy          2873 non-null object
Ambiencedivey           2864 non-null object
Ambiencehipster         2871 non-null object
Ambienceintimate        2873 non-null object
Ambienceromantic        2873 non-null object
Ambiencetouristy        2873 non-null object
Ambiencetrendy          2873 non-null object
Ambienceupscale         2866 non-null object
Attire                  2650 non-null object
Delivery                2777 non-null object
Good_For_Groups         3362 non-null object
Good_Forbreakfast       2508 non-null object
Good_Forbrunch          2503 non-null object
Good_Fordessert         2502 non-null object
Good_Fordinner          2507 non-null 

In [44]:
#I got rid of the hours of operation because they are not useful.
#ex. 'hoursWednesdayopen' and 'hoursWednesdayclose' have the same None response.
biz_wide.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
biz_wide.drop(['hoursFridayclose','hoursFridayopen', 'hoursMondayclose', 'hoursMondayopen',
                   'hoursSaturdayclose', 'hoursSaturdayopen', 'hoursSundayclose', 'hoursSundayopen', 
                   'hoursThursdayclose', 'hoursThursdayopen', 'hoursTuesdayclose', 'hoursTuesdayopen', 
                   'hoursWednesdayclose', 'hoursWednesdayopen'], axis=1, inplace=True)

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2. Constructing a "profile" for Las Vegas

---

Yelp is interested in building out what they are calling "profiles" for cities. They want you to start with just Las Vegas to see what a prototype of this would look like. Essentially, they want to know what makes Las Vegas distinct from the other four.

Use the data you have to predict Las Vegas from the other variables you have. You should not be predicting the city from any kind of location data or other data perfectly associated with that city (or another city).

You may use any classification algorithm you deem appropriate, or even multiple models. You should:

1. Build at least one model predicting Las Vegas vs. the other cities.
- Validate your model(s).
- Interpret and visualize, in some way, the results.
- Write up a "profile" for Las Vegas. This should be a writeup converting your findings from the model(s) into a human-readable description of the city.

*Research location data to find the city targets.*

1.Build at least one model predicting Las Vegas vs. the other cities.

In [57]:
formula = " ~ "+' + '.join([x for x in biz_wide.columns if not x in ['business_id','city']])+' -1'
print formula

 ~ attributes.Accepts Credit Cards + attributes.Accepts Insurance + attributes.Ages Allowed + attributes.Alcohol + attributes.Ambience.casual + attributes.Ambience.classy + attributes.Ambience.divey + attributes.Ambience.hipster + attributes.Ambience.intimate + attributes.Ambience.romantic + attributes.Ambience.touristy + attributes.Ambience.trendy + attributes.Ambience.upscale + attributes.Attire + attributes.BYOB + attributes.BYOB/Corkage + attributes.By Appointment Only + attributes.Caters + attributes.Coat Check + attributes.Corkage + attributes.Delivery + attributes.Dietary Restrictions.dairy-free + attributes.Dietary Restrictions.gluten-free + attributes.Dietary Restrictions.halal + attributes.Dietary Restrictions.kosher + attributes.Dietary Restrictions.soy-free + attributes.Dietary Restrictions.vegan + attributes.Dietary Restrictions.vegetarian + attributes.Dogs Allowed + attributes.Drive-Thru + attributes.Good For Dancing + attributes.Good For Groups + attributes.Good For.brea

In [99]:
#cutting and pasting the above (erasing attributes.) to retrieve the following:
formula = 'city ~ Accepts_Credit_Cards + Alcohol + Ambiencecasual + Ambienceclassy + Ambiencedivey + Ambiencehipster + Ambienceintimate + Ambienceromantic + Ambiencetouristy + Ambiencetrendy + Ambienceupscale + Attire + Delivery + Good_For_Groups + Good_Forbreakfast + Good_Forbrunch + Good_Fordessert + Good_Fordinner + Good_Forlatenight + Good_Forlunch + Good_for_Kids + Has_TV + Noise_Level + Outdoor_Seating + Parkinggarage + Parkinglot + Parkingstreet + Parkingvalet + Parkingvalidated + Price_Range + Take_out + Takes_Reservations + Waiter_Service + Wi_Fi + open - 1'

In [101]:
y, X = patsy.dmatrices(formula, data=biz_wide, return_type='dataframe')
y = np.ravel(y)
#Whyyy?? The error below says it's the variable name that is undefined, when I took out the variable
#it kept asking me to remove more and more until I didn't have any left.

NameError: name 'Take_out' is not defined

In [98]:
#Sklearn
ss = StandardScaler()
Xn = ss.fit_transform(X) 
'''
This is awful. Since the patsy formula is problematic, I couldn't define my variables y and X.
This means the rest of my attempts with Sklearn will not work.
I looked back at class notes and didn't see a problem with my code (I literally copy and pasted it).
I will put in the code I think is appropriate hoping I will at least get partial credit for the attempt
and for my logical reasoning.
'''


NameError: name 'X' is not defined

In [102]:
#K-neighbors classifier
params = {
    'n_neighbors':range(1,20),
    'weights':['uniform','distance']
}

knn = KNeighborsClassifier()
#GridsearchCv
knn_gs = GridSearchCV(knn, params, cv=4, verbose=1)

results = knn_gs.fit(Xn, y)

NameError: name 'Xn' is not defined

In [104]:
scores = cross_val_score(knn_gs, Xn, y, cv=20)
print scores
print "Cross-validated scores mean:", sum(scores)/len(scores)


predictions = cross_val_predict(knn_gs, Xn, y, cv=20)


accuracy = metrics.r2_score(y, predictions)

print "Cross-Predicted Accuracy:", accuracy

NameError: name 'Xn' is not defined

In [107]:
def plot_roc(true_y, predicted_y):

    # Get out the predicted probabilities for the X observation matrix
    # y_pp = knn_gs.predict_proba(Xn)

    # roc_curve returns the false positive rate and true positive rates as the threshold changes
    # takes in the y and the predicted probabilities of the positive class from your model.
    fpr, tpr, _ = roc_curve(true_y, predicted_y)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=[9,9])
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc, linewidth=4)
    plt.plot([0, 1], [0, 1], 'k--', linewidth=4)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)
    plt.title('Receiver operating characteristic curve', fontsize=20)
    plt.legend(loc="lower right")
    plt.show()
predicted = pd.DataFrame(results.best_estimator_.predict_proba(Xn))
plot_roc(y, predicted[1])

NameError: name 'results' is not defined

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. Different categories of ratings

---

Yelp is finally ready to admit that their rating system sucks. No one cares about the ratings, they just use the site to find out what's nearby. The ratings are simply too unreliable for people. 

Yelp hypothesizes that this is, in fact, because different people tend to give their ratings based on different things. They believe that perhaps some people always base their ratings on quality of food, others on service, and perhaps other categories as well. 

1. Do some users tend to talk about service more than others in reviews/tips? Divide up the tips/reviews into more "service-focused" ones and those less concerned with service.
2. Create two new ratings for businesses: ratings from just the service-focused reviews and ratings from the non-service reviews.
3. Construct a regression model for each of the two ratings. They should use the same predictor variables (of your choice). 
4. Validate the performance of the models.
5. Do the models coefficients differ at all? What does this tell you about the hypothesis that there are in fact two different kinds of ratings?

In [110]:
#Reviews SQL table
df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

In [111]:
df.head(2) #Let's see what we're dealing with

,user_id,review_id,votes_cool,business_id,votes_funny,stars,date,votes_useful,minutes_10,minutes_15,...,service_great,staff_friendly,super_friendly,sweet_potato,tasted_like,time_vegas,try_place,ve_seen,ve_tried,wait_staff
0,-ZWUL1xfFP-WA9iS87S8hQ,UOQie6_QAJ8hjceRYk5BXg,0,P7wKlbSRsoRrfH3DFOY08A,0,1,2014-09-08,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,lqXbPv5vlD5fu0TU_71AGQ,pb0yWYEyXqe80w-rf69NjA,0,oCA2OZcd_Jo_ggVmUx3WVw,0,1,2014-11-28,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 0 to 2499
Columns: 108 entries, user_id to wait_staff
dtypes: int64(102), object(6)
memory usage: 2.1+ MB


In [117]:
df.columns

Index([u'user_id', u'review_id', u'votes_cool', u'business_id', u'votes_funny',
       u'stars', u'date', u'votes_useful', u'minutes_10', u'minutes_15',
       ...
       u'service_great', u'staff_friendly', u'super_friendly', u'sweet_potato',
       u'tasted_like', u'time_vegas', u'try_place', u've_seen', u've_tried',
       u'wait_staff'],
      dtype='object', length=108)

In [ ]:
columns = list(df.columns.values)
print columns

#What things predict well whether a user is elite or not:
service_list = []
for rev in columns:
    if 'service' in rev: 
        service_list.append(rev)
    elif 'stars' in rev:
        service_list.append(rev)
    else:
        pass
    
        
print service_list

In [ ]:
y = df.stars
df = pd.DataFrame(df.data, columns=boston.feature_names)

In [ ]:
#EDA
full_data = df
full_data['stars'] = y

sns.pairplot(full_data)
plt.show()

In [ ]:
def correlation_heat_map(df):
    corrs = df.corr()

    # Set the default matplotlib figure size:
    fig, ax = plt.subplots(figsize=(12,12))

    # Generate a mask for the upper triangle (taken from seaborn example gallery)
    mask = np.zeros_like(corrs, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Plot the heatmap with seaborn.
    # Assign the matplotlib axis the function returns. This will let us resize the labels.
    ax = sns.heatmap(corrs, mask=mask, annot=True)

    # Resize the labels.
    ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=14, rotation=30)
    ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=14, rotation=0)

    # If you put plt.show() at the bottom, it prevents those useless printouts from matplotlib.
    plt.show()
    
correlation_heat_map(full_data)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print X_train.shape, y_train.shape
print X_test.shape, y_test.shape

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 4. Identifying "elite" users

---

Yelp, though having their own formula for determining whether a user is elite or not, is interested in delving deeper into what differentiates an elite user from a normal user at a broader level.

Use a classification model to predict whether a user is elite or not. Note that users can be elite in some years and not in others.

1. What things predict well whether a user is elite or not?
- Validate the model.
- If you were to remove the "counts" metrics for users (reviews, votes, compliments), what distinguishes an elite user, if anything? Validate the model and compare it to the one with the count variables.
- Think of a way to visually represent your results in a compelling way.
- Give a brief write-up of your findings.

In [ ]:
users.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

users.shape

In [ ]:
def count_elite(x):
    #Counts the number of years when user was elite
    return len(x.split())
    


users['elite_counts'] = users['elite'].map(count_elite)

In [ ]:
def years(x):
    #calculates how many years user has been yelping
    years = x.split('-')
    year = int(years[0])
    return (2016 - year)

users['yelping_years'] = users['yelping_since'].map(years)

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 5. Find something interesting on your own

---

You want to impress your superiors at Yelp by doing some investigation into the data on your own. You want to do classification, but you're not sure on what.

1. Create a hypothesis or hypotheses about the data based on whatever you are interested in, as long as it is predicting a category of some kind (classification).
2. Explore the data visually (ideally related to this hypothesis).
3. Build one or more classification models to predict your target variable. **Your modeling should include gridsearching to find optimal model parameters.**
4. Evaluate the performance of your model. Explain why your model may have chosen those specific parameters during the gridsearch process.
5. Write up what the model tells you. Does it validate or invalidate your hypothesis? Write this up as if for a non-technical audience.

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 6. ROC and Precision-recall

---

Some categories have fewer overall businesses than others. Choose two categories of businesses to predict, one that makes your proportion of target classes as even as possible, and another that has very few businesses and thus makes the target varible imbalanced.

1. Create two classification models predicting these categories. Optimize the models and choose variables as you see fit.
- Make confusion matrices for your models. Describe the confusion matrices and explain what they tell you about your models' performance.
- Make ROC curves for both models. What do the ROC curves describe and what do they tell you about your model?
- Make Precision-Recall curves for the models. What do they describe? How do they compare to the ROC curves?
- Explain when Precision-Recall may be preferable to ROC. Is that the case in either of your models?